<h1 style="display:none;">Test</h1>
<h1 style="display:none;">Test</h1>
<h1 style="display:none;">Test</h1>
<h1 style="line-height: 1.3">E6156: Topics in SW Engineering - Microservice and Cloud Applications.<br>REST Intro., PaaS/Elastic Beanstalk, Calling Cloud Service, Authentication/Authorization,</h1>

## Lecture Overview

- REST API


- Quick look at Open API Specification $-$ API Concepts


- Middleware Overview and Example


- Authentication and Authorization $-$ Review


- CloudFront, API Gateway

## Representational State Transfer (REST)

### Overview

- "Representational State Transfer (REST) is an architectural style that defines a set of constraints to be used for creating web services. Web Services that conform to the REST architectural style, or RESTful web services, provide interoperability between computer systems on the Internet. REST-compliant web services allow the requesting systems to access and manipulate textual representations of web resources by __using a uniform and predefined set of stateless operations.__ Other kinds of web services, such as SOAP web services, expose their own arbitrary sets of operations." \(Emphasis added\).(https://en.wikipedia.org/wiki/Representational_state_transfer)


- Non-RESTful applications surface service/domain specific operations, e.g.
    - ```open_account(...)```
    - ```transfer(...)```
    - ```check_balance(...)```
    

- The uniform, predefined REST operations are the HTTP Methods:
    - GET
    - PUT (or PATCH)
    - POST
    - DELETE
    
    
- These represent Create-Retrieve-Update-Delete operations on __resources__ identified by __URLs.__
    - POST is Create
    - GET is Retrieve
    - PUT (or PATCH) is Update
    - DELETE is Delete.
    
    
- __Note:__ People often confuse:
    - Remote procedure call/service invocation using HTTP
    - REST
    - They are not the same thing.
    
    
- The six core characteristics of the REST style are:
    1. Client–server architecture
    1. Statelessness
    3. Cacheability
    3. Layered system
    4. Code on demand (optional)
    6. Uniform interface


- You may also hear the term __Hypermedia As The Engine Of Application State (HATEOAS).__
    

### Client-Server Architecture

| <img src="./images/rest-client-server.jpg"> |
| :---: |
| __REST Client Server__ |

- "The client–server model is a distributed application structure that partitions tasks or workloads between the providers of a resource or service, called servers, and service requesters, called clients. Often clients and servers communicate over a computer network on separate hardware, but both client and server may reside in the same system." (https://en.wikipedia.org/wiki/Client%E2%80%93server_model)


- Concept is straightforward.

### Statelessness

- Statelessness is easy to misunderstand.


- The server _clearly_ has long-lived state information, e.g.
    - Account balances.
    - Customer contact information.
    - Product catalog information in a database.
    - etc.
    
    
- Client-Server interactions have two types of state:
    - Resource state
    - Conversation/Session
    
    
- "In computer science, in particular networking, a session is a temporary and interactive information interchange between two or more communicating devices, or between a computer and user." (https://en.wikipedia.org/wiki/Session_(computer_science))


| <img src="./images/session-state.jpeg"> |
| :---: |
| __Session/Conversation Start__ |

| <img src="./images/http_session.jpg"> |
| :---: |
| __HTTP Session__ |

- Database cursors are an example of conversation state.


- Example stateful "service" using cursors.

In [ ]:
import pymysql.cursors
import pandas as pd
import json


cnx = pymysql.connect(host='localhost',
                             user='dbuser',
                             password='dbuser',
                             db='lahman2017',
                             charset='utf8mb4',
                             cursorclass=pymysql.cursors.DictCursor)

cursor = cnx.cursor()


def get_by_last_name(lastName):
    cursor.execute("select * from people where nameLast=%s",(lastName));
    r = cursor.fetchone()
    return r

def get_next():
    r = cursor.fetchone()
    return r

- Example stateful client for stateful server.

In [ ]:
import cursor

first = cursor.get_by_last_name("Williams")

print("First = ", first)

done = False
while not done:
    next = cursor.get_next()
    if next is None or len(next) == 0:
        done = True
    else:
        print("Next = ", next)

- Statelessness in REST means that the server does not maintain conversation state.


- All requests from the client are complete and self-contained.


- The server _may_ return state to the client that the client must return on subsequent requests. $\Rightarrow$<br>The client maintains any conversation state the server requires.


| <img src="./images/rest_client.jpg">|
| :---: |
| [REST Self-Contained Messages](http://mrbool.com/rest-architectural-elements-and-constraints/29339) |

- Stateless server example

In [ ]:
import pymysql.cursors
import pandas as pd
import json


cnx = pymysql.connect(host='localhost',
                             user='dbuser',
                             password='dbuser',
                             db='lahman2017',
                             charset='utf8mb4',
                             cursorclass=pymysql.cursors.DictCursor)


def get_by_last_name(lastName, offset):
    cursor=cnx.cursor()
    q = "select * from people where nameLast=%s limit 1 offset %s"
    cursor.execute(q, (lastName, offset))
    r = cursor.fetchone()
    return r


- Statless client example

In [ ]:
import stateless_server


done = False
offset = 0

while not done:
    next = stateless_server.get_by_last_name("Williams", offset)
    if next is None or len(next) == 0:
        done = True
    else:
        print("Next = ", next)
        offset += 1


- Is there a concern about the client modifying or tinkering with the state information?


- Yes, and the server can encrypt the session state information to prevent tampering.


- Facebook example

| <img src="./images/facebook-request.jpg"> |
| :---: |
| __Sample Facebook Request__ |

- The Facebook request contains an encrypted _access token._ "In computer systems, an access token contains the security credentials for (...) identifies the user, the user's groups, the user's privileges, and, in some cases, a particular application." (https://en.wikipedia.org/wiki/Access_token)


- The response contains hashed and encrypted session state that MUST be returned to continue the conversational interaction.

### Cacheability

- Cacheability means exactly what the word implies. There may be several intermediaries between the client and server that caches a result.


- The intermediaries check the cache on a request and return the cached result without forwarding the request to the server.


| <img src="./images/cacheability.jpg"> |
| :---: |
| __Cacheability__ |

- The client and server can specify cache control headers in requests and responses.

| <img src="./images/cache_control_headers.jpeg"> |
| :---: |
| [Subset of Cache Control Directives](https://developer.mozilla.org/en-US/docs/Web/HTTP/Headers/Cache-Control)

### Layered System

- There may be many, many, many things including other microservices between the client and server. For example,
    - Firewalls.
    - Cache servers.
    - Middleware servers.
    - ... ...
    
    
- Part of designing and deploying microservices and cloud applications involves configuring or developing functionality that resides in various layers/intermediaries. We will see examples in the class.

### Code on Demand

- This primarily means that browsers (or devices) may be able to/have to download code to interact with server.


- JavaScript in the browser is the most common example, and we will do this in our projects.

### Uniform Interface

- Resource identification in requests:
    - URIs, and nothing else, identifies a resource.
    - Resources are representations (JSON, XML, ...), and the client is unaware of the underlying realization, e.g. relational database, some legacy application, ...
    
    
- Manipulation of Resources Through Representations: When a client holds a representation of a resource, including any metadata attached, it has enough information to modify or delete the resource on the server, provided it has permission to do so. There is no additional information or data necessary, for example in documentation or other services.


- Self-descriptive messages: Each message includes enough information to describe how to process the message.


- <u>Hypermedia as the Engine of Application State (HATEOAS):</u> Clients deliver state via body contents, query-string parameters, request headers and the requested URI (the resource name). Services deliver state to clients via body content, response codes, and response headers. __Responses contain links to related resources.__ Awareness of how to convert data into URIs is not necessary.


- This will become more clear as we build out our services. The best way to learn this vaguely explained concept is by implementing it.

## Customers API Overview

### Conceptual Model

- The [Open API Specification](https://swagger.io/specification/) provides a model (and some tools) for thinking about APIs and how to model/define them.


- [Open API Explorer](http://openapi-map.apihandyman.io/?version=3.0) is an interactive tool for understanding Open API definitions and the elements.


| <img src="./images/open_api.jpeg"> |
| :---: |
| [Open API Explorer](http://openapi-map.apihandyman.io/?version=3.0) |

- __Open API Swagger Demo__


- Open API is a systematic, thorough, complete, ... approach to publishing and collaborating on APIs.


- We do not need to do anything this systematic but the model is good to understand.


- The basic pattern focuses on paths.

```
/Customers
    GET /Customers?<query parameters>&fields=<list of fields>
    POST /Customers
    
/Customers/{id}
    GET fields=<list of fields>
    PUT
    DELETE
    
/Customers/{id}/<Some related resource>
```

- Once we have the API working, we will connect to the web page.


- __API Demo and code walkthrough__

| <img src="./images/postman_api_1.jpeg"> |
| :---: |
| __API Demo__ |


- Our application currently has the following paths:
    - /Customers
    - /Customers/{id}
    - /Register
    - /Login
    
    
- __Swagger Walkthrough and Demo__

| <img src="./images/swagger_1.jpeg"> |
| :---: |
| __Open API and Swagger__ |

## Middleware

### Concept Introduction

<img src="./images/middleware/Slide2.jpg">
<img src="./images/middleware/Slide3.jpg">
<img src="./images/middleware/Slide4.jpg">
<img src="./images/middleware/Slide5.jpg">

### Middleware Direction (for Course)

- We have seen an initial, ad hoc example in the login and authorization code.


- Please make sure you structure you authentication and authorization code following the model.


- We will start adding additional middleware capabilities, e.g. result caching in Redis, event notification.

## Authentication and Authorization $-$ Reminder

### Let's Register or Logon On

__Demo__

- The simplest and most basic approach is ```{userid, password}``` and will will start with it.


- What happens when we click on ```login```?

HTML

In [ ]:
<li ng-click="navMenu('login')" ng-class="getNavClass('login')" ng-show="!loginRegisterResult">
    <a class="nav-link" ng-click="doLogin()">Login</a>
</li>

JavaScript

In [ ]:
$scope.doLogin = function() {
    $("#loginModal").modal("show");
};

- What happens when the user complete the login form and submits? The controller
    1. Gets the data from the form fields.
    1. ```POSTs``` to ```/login``` or ```/register```
    1. Receives a success ```201``` or some kind of error.
    1. If success, saves the "token" in the browser application storage for subsequent requests.
    1. Updates state and may retrieve customer profile.

In [ ]:
$scope.driveLogin = function() {
        let req = null;
        let url = null;
        let op = null;

        if ($scope.register) {
            req = {
                lastName: $scope.lastName,
                firstName: $scope.firstName,
                email: $scope.lemail,
                pw: $scope.password
            };
            op = "register";
            url = getApiURL() + "/register";
        }
        else {
            req = {
                email: $scope.lemail,
                pw: $scope.password
            };
            op = "login";
            url = getApiURL() + "/login";
        };

        $http.post(url, req).then(
            function(result) {
                console.log("Result = " + JSON.stringify(result));
                let authorization = result.headers('authorization');
                $window.sessionStorage.setItem("credentials", JSON.stringify(authorization));
                $scope.loginRegisterResult = true;
                $scope.loginRegisterMessage = "Success. Registered/Logged on. Click close";
                getCustomerInfo(result.data)
            },
            function(error) {
                console.log("Result = " + JSON.stringify(result));
                $scope.loginRegisterMessage = "Failed. Close and try again."
                $scope.loginRegisterResult = true;
            }
        );

    };

- We keep the returned token in _browser session storage_ and send on subsequent requests.


- "The sessionStorage property allows you to access a session Storage object for the current origin. sessionStorage is similar to Window.localStorage; the only difference is while data stored in localStorage has no expiration set, data stored in sessionStorage gets cleared when the page session ends. A page session lasts for as long as the browser is open and survives over page reloads and restores." (https://developer.mozilla.org/en-US/docs/Web/API/Window/sessionStorage) 


- There is also longer lived ```local storage.```

| <img src="./images/session_storage.jpeg"> |
| :---: |
| __Session Storage__ |

### Server Side Code


- There is a _business logic layer object/module_ that implements ```login()``` and ```register().```


- Implements functionality by using ```customer business object.```


- Code walkthrough.

## Web, HTTPS, Single Site Image

### CloudFront

<img src="./images/cloud-front/Slide01.jpg">
<img src="./images/cloud-front/Slide02.jpg">

- All enterprise sites having something like CloudFront to provide
    - Acceleration, scalability, ...
    - Single site image in front of multiple sources of content and APIs.
    
    

- CloudFront walk through.

### Web/Microservice Security Introduction

<img src="./images/cloud-front/Slide03.jpg">
<img src="./images/cloud-front/Slide04.jpg">
<img src="./images/cloud-front/Slide05.jpg">

<img src="./images/cloud-front/Slide06.jpg">
<img src="./images/cloud-front/Slide07.jpg">
<img src="./images/cloud-front/Slide08.jpg">

<img src="./images/cloud-front/Slide09.jpg">
<img src="./images/cloud-front/Slide10.jpg">
<img src="./images/cloud-front/Slide11.jpg">

<img src="./images/cloud-front/Slide12.jpg">
<img src="./images/cloud-front/Slide13.jpg">
<img src="./images/cloud-front/Slide14.jpg">

<img src="./images/cloud-front/Slide15.jpg">
<img src="./images/cloud-front/Slide16.jpg">
<img src="./images/cloud-front/Slide17.jpg">
<img src="./images/cloud-front/Slide18.jpg">